In [ ]:
import urllib.parse
import urllib.request
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

import pandas as pd
from tqdm import tqdm

In [ ]:
# https://lila.science/public/lila_image_urls_and_labels.csv.zip
LILA_IMAGES_URLS_AND_LABELS = 'data/lila_image_urls_and_labels.csv'
LILA_COYOTE_IMAGES_URLS_AND_LABELS = 'data/lila_coyote_image_urls_and_labels.csv'

In [ ]:
with open(LILA_IMAGES_URLS_AND_LABELS) as i, open(LILA_COYOTE_IMAGES_URLS_AND_LABELS, 'w') as o:
    o.write(i.readline())
    for line in i:
        if 'coyote' in line:
            o.write(line)


In [ ]:
df = pd.read_csv(LILA_COYOTE_IMAGES_URLS_AND_LABELS)
df

In [ ]:
def download(url: str):
    filename = f"data{urllib.parse.urlparse(url).path}"
    path = Path(filename)
    path.parent.mkdir(parents=True, exist_ok=True)
    if path.exists():
        return
    retries = 0
    while True:
        try:
            urllib.request.urlretrieve(url, filename)
            break
        except Exception as e:
            retries += 1
            if retries == 3:
                raise e


In [ ]:
def main():
    with ThreadPoolExecutor(max_workers=256) as executor:
        results = list(
            tqdm(executor.map(download, df['url']), total=len(df['url']))
        )

main()
    